In [1]:
import csv
import time
from confluent_kafka import Producer

In [2]:
bootstrap_server = "kafka:9092"
kafka_ES_storage = "hwinfo"
# kafka_RT_dashboard = "hwinfo_logs_RT" # do not touch or uncomment

In [3]:
producer = Producer({"bootstrap.servers": bootstrap_server})

In [4]:
hwinfo_logs_file = 'test1.csv'

In [5]:
def delivery_report(err, msg):
    if err is not None:
        print(f"failure in delivering messages: {err}")

In [6]:
# def send_data(hwinfo_csv):
#     last_pos = 0
#     while True:
#         with open(hwinfo_csv, mode = 'r', encoding='ISO-8859-1') as file:
#             file.seek(last_pos)
#             reader = csv.reader(file)
#             for row in reader:
#                 producer.produce(kafka_ES_storage, value = str(row).encode('utf-8'), callback = delivery_report)
#                 producer.produce(kafka_RT_dashboard, value = str(row).encode('utf-8'), callback = delivery_report)
#                 # print(f"sent row data to kakfa: {row}")
#             last_pos = file.tell()
#         producer.poll(1)
#         time.sleep(1)

In [7]:
def send_data(hwinfo_csv):
    last_pos = 0  # Track last read position
    
    while True:
        try:
            with open(hwinfo_csv, mode="r", encoding="ISO-8859-1") as file:
                file.seek(last_pos)  # Move to last read position
                reader = csv.reader(file)

                for row in reader:
                    message = str(row).encode("utf-8")

                    # Send to both Kafka topics
                    producer.produce(kafka_ES_storage, value = str(row).encode('utf-8'), callback = delivery_report)
                    
                    
                    # this thing is for spark only DO NOT UNCOMMENT
                    # producer.produce(kafka_RT_dashboard, value = str(row).encode('utf-8'), callback = delivery_report) 

                last_pos = file.tell()  # Update last read position

            producer.poll(0)  # Process Kafka events
            producer.flush()   # Ensure messages are sent
            time.sleep(1)      # Reduce CPU usage

        except Exception as e:
            print(f"Error in reading log file: {e}")
            time.sleep(2)  # Avoid crash-looping


In [8]:
send_data(hwinfo_logs_file)

KeyboardInterrupt: 